## Jax Level 0 - Chapter 1
### Part 2 - Data Owner responds to the requests

Link to the original Jax tutorial: https://jax.readthedocs.io/en/latest/jax-101/01-jax-basics.html

In [1]:
# Import the necessary libraries
import syft as sy
sy.requires(">=0.8-beta")

/home/teo/anaconda3/envs/PySyft/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo


✅ The installed version of syft==0.8.0b6 matches the requirement >=0.8b0


In [2]:
# Check domain notifications
node = sy.orchestra.launch(name="test-domain-1")
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_client.notifications

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]


,type,id,subject,status,created_at,linked_obj
0,syft.core.node.new.messages.Message,b3e3a2bf693644e69938b701dc3a9da0,Approval Request,MessageStatus.UNDELIVERED,2023-04-14 09:58:38,<<class 'syft.core.node.new.request.Request'>:...
1,syft.core.node.new.messages.Message,35d24f0e1c33482f9153893d89db287e,Approval Request,MessageStatus.UNDELIVERED,2023-04-14 09:58:45,<<class 'syft.core.node.new.request.Request'>:...
2,syft.core.node.new.messages.Message,357d180747dd45639481c6bf2181de9e,Approval Request,MessageStatus.UNDELIVERED,2023-04-14 09:58:53,<<class 'syft.core.node.new.request.Request'>:...
3,syft.core.node.new.messages.Message,374a0d7ad0aa47e0a0cbe44e0063dcba,Approval Request,MessageStatus.UNDELIVERED,2023-04-14 09:58:38,<<class 'syft.core.node.new.request.Request'>:...
4,syft.core.node.new.messages.Message,c20f2d4fd8254b43b2cd76236e70fe84,Approval Request,MessageStatus.UNDELIVERED,2023-04-14 09:58:46,<<class 'syft.core.node.new.request.Request'>:...


In [3]:
# Get messages from domain
messages = domain_client.api.services.messages.get_all()
messages


,type,id,subject,status,created_at,linked_obj
0,syft.core.node.new.messages.Message,b3e3a2bf693644e69938b701dc3a9da0,Approval Request,MessageStatus.UNDELIVERED,2023-04-14 09:58:38,<<class 'syft.core.node.new.request.Request'>:...
1,syft.core.node.new.messages.Message,35d24f0e1c33482f9153893d89db287e,Approval Request,MessageStatus.UNDELIVERED,2023-04-14 09:58:45,<<class 'syft.core.node.new.request.Request'>:...
2,syft.core.node.new.messages.Message,357d180747dd45639481c6bf2181de9e,Approval Request,MessageStatus.UNDELIVERED,2023-04-14 09:58:53,<<class 'syft.core.node.new.request.Request'>:...
3,syft.core.node.new.messages.Message,374a0d7ad0aa47e0a0cbe44e0063dcba,Approval Request,MessageStatus.UNDELIVERED,2023-04-14 09:58:38,<<class 'syft.core.node.new.request.Request'>:...
4,syft.core.node.new.messages.Message,c20f2d4fd8254b43b2cd76236e70fe84,Approval Request,MessageStatus.UNDELIVERED,2023-04-14 09:58:46,<<class 'syft.core.node.new.request.Request'>:...


In [4]:
# Fetch the dataset
results = domain_client.api.services.dataset.get_all()
dataset = results[0]
mock = dataset.assets[0].data.syft_action_data
xs = dataset.assets[1].data.syft_action_data
ys = dataset.assets[2].data.syft_action_data

In [8]:
mock.data.syft_action_data

array([1., 2., 3., 4.])

In [ ]:
possible_kwargs = {
    "func": {},
    "func_timeit": {},
    "func_sum_of_squares" : {"x": mock},
    "func_sum_squared_error" : {"x": mock},
    "func_sum_squared_error_with_aux" : {"x": mock},
    "func_naive_modify" : {"x": mock},
    "func_jax_modify" : {},
    "training_loop" : {'xs': xs.mock, 'ys': ys.mock}
}


In [ ]:
# Approve each request and run the user codes 
for message in messages:    
    request = message.link
    func = request.changes[0].link
    func_name = func.service_func_name
    print(func_name)
    user_func = func.unsafe_function
    kwargs = possible_kwargs[func_name]
    real_result = user_func(**kwargs)
    print(real_result)
    result = request.accept_by_depositing_result(real_result)
    print(result)

func
syft.core.node.new.user_code.UserCodeExecutionResult


execute_byte_code


message="<class 'syft.core.node.new.action_object.AnyActionObject'> saved"
syft.core.node.new.user_code.UserCodeExecutionResult
message='Request d543efaf76b84caab1ae9e4e88d1bbf4 changes applied'
func_sum_squared_error_with_aux
None


execute_byte_code failed Argument 'syft.core.node.new.dataset.Asset' of type <class 'syft.core.node.new.dataset.Asset'> is not a valid JAX type.


message="<class 'syft.core.node.new.action_object.AnyActionObject'> saved"
None
message='Request e70c44d586a745ae9b0e51982feeb20d changes applied'
func_naive_modify
syft.core.node.new.user_code.UserCodeExecutionResult


...

message="<class 'syft.core.node.new.action_object.AnyActionObject'> saved"
None
message='Request 584c7f1673d4415cb28a97f3bb5379ea changes applied'
func_jax_modify
syft.core.node.new.user_code.UserCodeExecutionResult


execute_byte_code


message="<class 'syft.core.node.new.action_object.AnyActionObject'> saved"
syft.core.node.new.user_code.UserCodeExecutionResult
message='Request 565c64efa3dd4a7885279310e92289f5 changes applied'
func_sum_squared_error
None


execute_byte_code failed Argument 'syft.core.node.new.dataset.Asset' of type <class 'syft.core.node.new.dataset.Asset'> is not a valid JAX type.


message="<class 'syft.core.node.new.action_object.AnyActionObject'> saved"
None
message='Request 3b1cd814bd6e4ae083a80b85b86f842a changes applied'


Go to Part 3!